In [1]:
%pip install -qU langchain-community beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install -qU langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [24]:
import bs4
import re

from bs4 import BeautifulSoup


def bs4_extractor(html: str) -> str:
    soup = BeautifulSoup(html, "lxml")
    return re.sub(r"\n\n+", "\n\n", soup.text).strip()

from langchain_community.document_loaders import RecursiveUrlLoader

loader = RecursiveUrlLoader(
    "https://docs.xbim.net/",
    extractor=bs4_extractor,
    max_depth=5,
    # use_async=False,
    # extractor=None,
    # metadata_extractor=None,
    # exclude_dirs=(),
    # timeout=10,
    # check_response_status=True,
    # continue_on_failure=True,
    # prevent_outside=True,
    # base_url=None,
    # ...
)
docs = []
async for doc in loader.alazy_load():
    docs.append(doc)

print(len(docs))

32


In [25]:
doc = docs[0]
print(f"{doc.metadata}\n")

{'source': 'https://docs.xbim.net/', 'content_type': 'text/html; charset=utf-8', 'title': 'xbim toolkit', 'description': 'xbim toolkit is an Open Source toolkit for IFC and BIM. You can use it to do anything with IFC. It implements IFC2x3 and IFC4 and allows to develop SW agnostic to IFC version and file format.', 'language': 'en'}



In [26]:
print(doc.page_content[:500].strip())

xbim toolkit

Toggle navigation

Quick Start

Docs

Examples 

Examples list

Basic 

Basic Model Operations

Watch the log!

Proper Wall in 3D

STEP to XML and Back Again

Creating wexBIM file

Using LINQ for Optimal Performance

Excel Space Report from IFC

Spatial hierarchy

Advanced 

Federating Models

Insert Copy Function

Change Log Creation

Web UI  

Hello Building

Safe Hello Building

Eventful Building

Building Seen from Everywhere

Colourful Building

Pinpoint the View

Informationa


In [3]:
import getpass
import os

os.environ["AZURE_OPENAI_API_KEY"] = getpass.getpass("Enter API key for Azure: ")

Enter API key for Azure:  ········


In [6]:
from langchain_openai import AzureOpenAIEmbeddings

embeddings = AzureOpenAIEmbeddings(
    azure_endpoint="https://oai-bim42-test-fr-ai.openai.azure.com",
    azure_deployment="text-embedding-3-small",
    openai_api_version="2025-01-01-preview",
)

In [27]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

305

In [28]:
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import AzureOpenAIEmbeddings

vector_store = InMemoryVectorStore.from_documents(all_splits, embeddings)

In [40]:
results = vector_store.similarity_search_with_score("can digest models")
print()
print(f"{len(results)} results found\n")
doc, score = results[0]
print(f"First result Score: {score}\n")
print(f"Content:\n{doc.page_content}\n")
print(doc.metadata)


4 results found

First result Score: 0.35280009177534666

Content:
That is scalable in a way but far from being optimal or cost efficient.
Based on all our findings we have developed a new cloud platform xbim flex, which uses xbim toolkit
under the hood and provides access to building data in a completely scalable way as a commercial service. It can process any number 
of models at any time and can digest models which make many other platforms choke. If you are interested, 
download the Flex Revit Add-In here or get in touch at info@xbim.net.
Our first application on the platform is helping people to communicate around BIM models using open standards 
in the cloud: www.xbim.net. We have also found and fixed several issues with IFC production, 
you can download our Revit  Add-in for free and see how the platform works at www.xbim.net/download-addin.
Web application for communication around models:

{'source': 'https://docs.xbim.net/cloud.html', 'content_type': 'text/html; charset=utf-8